# Introduzione

Obiettivo di questo notebook: testare il training degli algoritmi Implicit Q-Learning, Conservative Q-Learning, Behavior Cloning, TD3 con Behavior Cloning sull dataset `D4RL/pen/expert-v2`.

Il task "pen" richiede a una mano robotica (Adroit Hand) di manipolare una penna per portarla in una certa posizione nello spazio.

# Caricamento del dataset

In [1]:
import minari
import time
import numpy as np
from d3rlpy.algos import IQLConfig, CQLConfig, BCConfig, TD3PlusBCConfig, BCQConfig, AWACConfig
from d3rlpy.datasets import MDPDataset
from d3rlpy.constants import ActionSpace
from d3rlpy.metrics import EnvironmentEvaluator

In [14]:
dataset = minari.load_dataset("D4RL/hammer/expert-v2")

In [15]:
print("Episodi totali:", dataset.total_episodes)
print("Spazio osservazioni:", dataset.observation_space)
print("Spazio azioni:", dataset.action_space)

Episodi totali: 5000
Spazio osservazioni: Box(-inf, inf, (46,), float64)
Spazio azioni: Box(-1.0, 1.0, (26,), float32)


In [16]:
episode = next(dataset.iterate_episodes())
print(episode)

#print(f"Osservazioni: \n{episode.observations[0]}")
#print(f"Actions: \n{episode.actions[0]}")
#print(f"Rewards: \n{episode.rewards[0]}")
#print(f"Terminations: \n{episode.terminations[0]}")

EpisodeData(id=0, total_steps=200, observations=ndarray of shape (201, 46) and dtype float64, actions=ndarray of shape (200, 26) and dtype float32, rewards=ndarray of 200 floats, terminations=ndarray of 200 bools, truncations=ndarray of 200 bools, infos=dict with the following keys: ['success'])


The task to be completed consists on repositioning the blue pen to match the orientation of the green target. The base of the hand is fixed. The target is also randomized to cover all configurations. The task will be considered successful when the orientations match within tolerance

# Preparazione dataset

d3rlpy si aspetta che il dataset sia composto da transizioni, in cui ogni elemento contiene uno stato, un’azione, una ricompensa, lo stato successivo e un flag terminale, tutti allineati in modo che lo stato e l’azione alla posizione i corrispondano alla transizione verso lo stato alla posizione i+1. A tal fine, la libreria mette a disposizione la classe MDPDataset, che consente di creare facilmente un oggetto dataset nel formato richiesto. All'interno del dataset non c'è la distinzione in episodi, tutti gli step sono uniti in un unico array.

In [5]:
observations = []
actions = []
rewards = []
terminals = []

for episode in dataset.iterate_episodes():
    # si rimuove l'ultimo elemento, in quanto non ha una successiva azione associata
    obs = episode.observations[:-1]
    actions_ep = episode.actions
    rewards_ep = episode.rewards
    dones = np.array(episode.terminations) | np.array(episode.truncations)

    observations.append(obs)
    actions.append(actions_ep)
    rewards.append(rewards_ep)
    terminals.append(dones)

# ora observations è un array di 4958 array (episodi) di 100 array circa (step) di array (osservazioni). stesso discorso per gli altri

# si uniscono gli array in modo da avere, per ogni step del dataset osservazioni, azione, reward, terminali
observations = np.concatenate(observations)
actions = np.concatenate(actions)
rewards = np.concatenate(rewards)
terminals = np.concatenate(terminals)

# ora observations è un array di 499206 (step in tutto il dataset) di array (osservazioni) . stesso discorso per gli altri
print(observations.shape)
print(actions.shape)
print(rewards.shape)
print(terminals.shape)

d3_dataset = MDPDataset(observations, actions, rewards, terminals, action_space = ActionSpace.CONTINUOUS)

(499206, 45)
(499206, 24)
(499206,)
(499206,)
2025-04-29 11:48.12 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(24,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(45,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-04-29 11:48.12 [info     ] Action size has been automatically determined. action_size=24


# Implicit Q-Learning

In [ ]:
iql = IQLConfig().create(device="cpu")

In [ ]:
iql.build_with_dataset(d3_dataset)

In [ ]:
env = dataset.recover_environment()

iql.fit(
    dataset=d3_dataset,
    n_steps=10000,
    n_steps_per_epoch=1000,
    evaluators={"env": EnvironmentEvaluator(env)},
)

In [ ]:
env = dataset.recover_environment(render_mode="human", camera_id=2)
obs, _ = env.reset()
done = False
total_reward = 0

for _ in range(1000):
    action = iql.predict(obs[None])[0]
    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward
    if terminated:
        break

env.close()
print(f"Reward totale: {total_reward}")

# Conservative Q-Learning

In [ ]:
cql = CQLConfig().create(device="cpu")

In [ ]:
cql.build_with_dataset(d3_dataset)

In [ ]:
env = dataset.recover_environment()

cql.fit(
    dataset=d3_dataset,
    n_steps=10000,
    n_steps_per_epoch=1000,
    evaluators={"env": EnvironmentEvaluator(env)},
)

In [ ]:
env = dataset.recover_environment(render_mode="human", camera_id=2)
obs, _ = env.reset()
done = False
total_reward = 0

for _ in range(1000):
    action = cql.predict(obs[None])[0]
    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward
    if terminated:
        break

env.close()
print(f"Reward totale: {total_reward}")

# Behavior Cloning

In [ ]:
bc = BCConfig().create(device="cpu")

In [ ]:
bc.build_with_dataset(d3_dataset)

In [ ]:
env = dataset.recover_environment()

bc.fit(
    dataset=d3_dataset,
    n_steps=10000,
    n_steps_per_epoch=1000,
    evaluators={"env": EnvironmentEvaluator(env)},
)

In [ ]:
env = dataset.recover_environment(render_mode="human", camera_id=2)
obs, _ = env.reset()
done = False
total_reward = 0

for _ in range(1000):
    action = bc.predict(obs[None])[0]
    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward
    if terminated:
        break

env.close()
print(f"Reward totale: {total_reward}")

# TD3 + BC

In [ ]:
td3bc = TD3PlusBCConfig().create(device="cpu")

In [ ]:
td3bc.build_with_dataset(d3_dataset)

In [ ]:
env = dataset.recover_environment()

td3bc.fit(
    dataset=d3_dataset,
    n_steps=10000,
    n_steps_per_epoch=1000,
    evaluators={"env": EnvironmentEvaluator(env)},
)

In [ ]:
env = dataset.recover_environment(render_mode="human", camera_id=2)
obs, _ = env.reset()
done = False
total_reward = 0

for _ in range(1000):
    action = td3bc.predict(obs[None])[0]
    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward
    if terminated:
        break

env.close()
print(f"Reward totale: {total_reward}")

# BCQ

In [ ]:
bcq = BCQConfig().create(device="cpu")

In [ ]:
bcq.build_with_dataset(d3_dataset)

In [ ]:
env = dataset.recover_environment()

bcq.fit(
    dataset=d3_dataset,
    n_steps=10000,
    n_steps_per_epoch=1000,
    evaluators={"env": EnvironmentEvaluator(env)},
)

In [ ]:
env = dataset.recover_environment(render_mode="human", camera_id=2)
obs, _ = env.reset()
done = False
total_reward = 0

for _ in range(1000):
    action = bcq.predict(obs[None])[0]
    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward
    if terminated or truncated:
        break

env.close()
print(f"Reward totale: {total_reward}")

# AWAC

In [6]:
awac = AWACConfig().create(device="cpu")

In [7]:
awac.build_with_dataset(d3_dataset)

In [8]:
env = dataset.recover_environment()

awac.fit(
    dataset=d3_dataset,
    n_steps=10000,
    n_steps_per_epoch=1000,
    evaluators={"env": EnvironmentEvaluator(env)},
)

2025-04-29 11:48.25 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(45,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(24,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=24)
2025-04-29 11:48.25 [warning  ] Skip building models since they're already built.
2025-04-29 11:48.25 [info     ] Directory is created at d3rlpy_logs/AWAC_20250429114825
2025-04-29 11:48.25 [info     ] Parameters                     params={'observation_shape': [45], 'action_size': 24, 'config': {'type': 'awac', 'params': {'batch_size': 1024, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'actor_learning_rate': 0.0003, 'critic_learning_rate': 0.0003, 'actor_optim_factory': {'type': 'adam', 'par

Epoch 1/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-29 11:48.43 [info     ] AWAC_20250429114825: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.0064957036972045894, 'time_algorithm_update': 0.010676538467407227, 'critic_loss': 554.7457459411621, 'actor_loss': 425110.322421875, 'temp': 0.0, 'temp_loss': 0.0, 'time_step': 0.017215315580368044, 'env': 562.5663216534314} step=1000
2025-04-29 11:48.43 [info     ] Model parameters are saved to d3rlpy_logs/AWAC_20250429114825/model_1000.d3


Epoch 2/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-29 11:49.00 [info     ] AWAC_20250429114825: epoch=2 step=2000 epoch=2 metrics={'time_sample_batch': 0.006325634002685547, 'time_algorithm_update': 0.010734166860580444, 'critic_loss': 546.8411540832519, 'actor_loss': 207324.41248828126, 'temp': 0.0, 'temp_loss': 0.0, 'time_step': 0.017101824045181273, 'env': 540.1014425113003} step=2000
2025-04-29 11:49.00 [info     ] Model parameters are saved to d3rlpy_logs/AWAC_20250429114825/model_2000.d3


...

Epoch 9/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-29 11:51.03 [info     ] AWAC_20250429114825: epoch=9 step=9000 epoch=9 metrics={'time_sample_batch': 0.006766695022583008, 'time_algorithm_update': 0.011246136665344238, 'critic_loss': 12170.222025878906, 'actor_loss': 13423.196507347107, 'temp': 0.0, 'temp_loss': 0.0, 'time_step': 0.018054604053497316, 'env': 1530.4305660160392} step=9000
2025-04-29 11:51.03 [info     ] Model parameters are saved to d3rlpy_logs/AWAC_20250429114825/model_9000.d3


Epoch 10/10:   0%|          | 0/1000 [00:00<?, ?it/s]

2025-04-29 11:51.21 [info     ] AWAC_20250429114825: epoch=10 step=10000 epoch=10 metrics={'time_sample_batch': 0.006686381101608276, 'time_algorithm_update': 0.010797379970550537, 'critic_loss': 15307.128662597655, 'actor_loss': 9125.657841182709, 'temp': 0.0, 'temp_loss': 0.0, 'time_step': 0.017527610540390013, 'env': 1781.371999825909} step=10000
2025-04-29 11:51.21 [info     ] Model parameters are saved to d3rlpy_logs/AWAC_20250429114825/model_10000.d3


[(1,
  {'time_sample_batch': 0.0064957036972045894,
   'time_algorithm_update': 0.010676538467407227,
   'critic_loss': 554.7457459411621,
   'actor_loss': 425110.322421875,
   'temp': 0.0,
   'temp_loss': 0.0,
   'time_step': 0.017215315580368044,
   'env': 562.5663216534314}),
 (2,
  {'time_sample_batch': 0.006325634002685547,
   'time_algorithm_update': 0.010734166860580444,
   'critic_loss': 546.8411540832519,
   'actor_loss': 207324.41248828126,
   'temp': 0.0,
   'temp_loss': 0.0,
   'time_step': 0.017101824045181273,
   'env': 540.1014425113003}),
 (3,
  {'time_sample_batch': 0.006434033870697022,
   'time_algorithm_update': 0.010788160800933838,
   'critic_loss': 1128.686857849121,
   'actor_loss': 115807.14467773438,
   'temp': 0.0,
   'temp_loss': 0.0,
   'time_step': 0.017263113498687744,
   'env': 85.39976027247272}),
 (4,
  {'time_sample_batch': 0.006201712608337402,
   'time_algorithm_update': 0.010133864879608155,
   'critic_loss': 2037.0081600952149,
   'actor_loss': 75

In [13]:
env = dataset.recover_environment(render_mode="human", camera_id=2)
obs, _ = env.reset()
done = False
total_reward = 0

for _ in range(1000):
    action = awac.predict(obs[None])[0]
    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward
    if terminated or truncated:
        break

env.close()
print(f"Reward totale: {total_reward}")

Reward totale: 3628.426871523513
